# Legacy DSA
(it is a legacy algorithm because it does not use Elliptic Curves)

In [133]:
from sympy.ntheory import isprime

from cryptography.hazmat.primitives import hashes
from cryptography.hazmat.primitives.asymmetric import dsa
from cryptography.hazmat.primitives.asymmetric import utils #decode_dss_signature

DSA works over a modular group $Z_p$, with $p$ prime.

The private key is $x \in [0,q)$.

The public key is $y = g^x \mod p$

In [154]:
private_key = dsa.generate_private_key( key_size=1024 )
public_key = private_key.public_key()
chosen_hash = hashes.SHA1()

print("Private key is: %d" % private_key.private_numbers().x)
print("Public key is : %s" % public_key.public_numbers().y)

Private key is: 961379271991097943513107905027321113521197974689
Public key is : 48477392754393955024438228634301590594143208439654261402727738814431244365383291877571586040764824333019609272369268314232735617882729981394183302519262765414636339542136111572084592255503665718126044923788418771871782266374209405133883754338260355467090455976840997696455985413893208152786415108744976526421


In [155]:
# parameters
# The group is Zp
p = private_key.private_numbers().public_numbers.parameter_numbers.p
# The order is q
q = private_key.private_numbers().public_numbers.parameter_numbers.q
# The group generator is g
g = private_key.private_numbers().public_numbers.parameter_numbers.g
# The private key is x
x = private_key.private_numbers().x
# The public key is y
y = private_key.private_numbers().public_numbers.y

In [156]:
# Is q prime?
isprime(q)

True

In [157]:
# Does q divide p-1?
(p-1) % q

0

In [158]:
# is y = g^x mod p?
pow(g,x,p) == y

True

## DSA signature
To sign we need a random nonce $k \in [1,q)$.

The signature is the pair $(r,s)$ with
$$ r = (g^k \mod p) \mod q $$
$$ s = (k^{-1} (h(m) + xr) ) \mod q$$

In [159]:
message = b'message'
sig = private_key.sign(message,chosen_hash)

# print the signature as hex
sig.hex()

'302d021500a5f5b8c0a8947ffa138d628f07e180467ac4d77e021414f5e089dd521b0a171e07e6c412f52aadc6dc9b'

In [160]:
(r,s)=utils.decode_dss_signature(sig)
(r,s)

(947463253978480775779753407201527889614952781694,
 119663058055035479453519114597518445521616166043)

## Can we verify the signature?

We need
$$ u_1 = h(m) s^{-1} \mod q $$
$$ u_2 = r s^{-1} \mod q $$
The signature is verified if
$$ (g^{u_1}y^{u_2} \mod p) \mod q= r $$

In [161]:
hasher = hashes.Hash(chosen_hash)
hasher.update(message)
digest = hasher.finalize()
hm = int.from_bytes(digest,"big")

In [166]:
u1 = (hm*pow(s,-1,q)) % q
u2 = (r*pow(s,-1,q)) % q
( (pow(g,u1,p)*pow(y,u2,p)) % p)  % q

947463253978480775779753407201527889614952781694

In [163]:
r

947463253978480775779753407201527889614952781694

In [168]:
# Now we use the library function for signature verification
try:
    public_key.verify(sig,message,chosen_hash)
    print("OK")
except:
    print("KO")

OK
